本文介绍牛顿迭代法（Newton's method for finding roots）求解方程的近似解。牛顿在 1664 年发明了这个方法。具体的任务是，对于在 $[a,b]$ 上连续且单调的函数 $f(x)$ ，求 $f(x)=0$ 的近似解。

## 算法描述

初始时我们从给定的 $f(x)$ 和一个粗略的近似解 $x_0$ 开始。

假设我们目前的近似解是 $x_i$ ，那么为了得到更优的近似解，我们画出与 $f(x)$ 切于点 $(x_0,f(x_0))$ 的直线 $l$ ，将 $l$ 与 $x$ 轴的交点横坐标记为 $x_{i+1}$ 。并重复这个迭代的过程。

上述过程可以表示为以下递推式：

$$
 x_{i+1} = x_i - \frac{f(x_i)}{f^\prime(x_i)}
$$

![](https://p.ssl.qhimg.com/t018a304e49e41ca606.jpg)

直观地说，如果 $f(x)$ 比较平滑，那么随着迭代次数的增加， $x_i$ 会越来越逼近方程的解。

牛顿迭代法的收敛率是平方级别的，这意味着每次迭代后近似解的精确数位会翻倍。

## 求解平方根

我们尝试用牛顿迭代法求解平方根。设 $f(x)=x^2-n$ ，这个方程的近似解就是 $\sqrt{n}$ 的近似解。于是我们得到

$$
x_{i+1}=x_i-\frac{x_i^2-n}{2x_i}=\frac{x_i+\frac{n}{x_i}}{2}
$$

In [2]:
def sqrt_newton(n):
    eps=1e-15
    x=1
    while True:
        nx=(x+n/x)/2
        if abs(x-nx)<eps:
            break
        x=nx
    return x

sqrt_newton(3)

1.7320508075688772

## 求解整数平方根

尽管我们可以调用 `sqrt()` 函数来获取平方根的值，但我们还是讲一下牛顿迭代法的变种算法，用于求解 $x^2\le n$ 的 $x$ 的最大整数解。我们仍然考虑一个类似于牛顿迭代的过程，但需要在边界条件上稍作修改。如果 $x$ 在迭代的过程中上一次迭代值得近似解变小，而这一次迭代使得近似解变大，那么我们就不进行这次迭代，退出循环。

In [14]:
def isqrt_newton(n):
    x=1
    decreased=False
    while True:
        nx=int((x+n/x)/2)
        if x==nx or nx>x and decreased:
            break
        decreased=nx<x
        x=nx
    return x
isqrt_newton(3)

1

## 高精度平方根

最后考虑高精度的牛顿迭代法。迭代的方法是不变的，但这次我们需要关注初始时近似解的设置，即 $x_0$ 的值。由于需要应用高精度的数一般都非常大，因此不同的初始值对于算法效率的影响也很大。一个自然的想法就是考虑 $x_0=2^{\left\lfloor\frac{1}{2}\log_2n\right\rfloor}$ ，这样既可以快速计算出 $x_0$ ，又可以较为接近平方根的近似解。

In [29]:
def isqrt_newton_h(n):
    x=1<<((len(bin(n))-2)>>1)
    decreased=False
    while True:
        nx=(x+n//x)>>1
        if x==nx or nx>x and decreased:
            break
        decreased=nx<x
        x=nx
    return x
isqrt_newton_h(2**1000)

3273390607896141870013189696827599152216642046043064789483291368096133796404674554883270092325904157150886684127560071009217256545885393053328527589376

[UVa 10428 The Roots](http://uva.onlinejudge.org/index.php?option=com_onlinejudge&Itemid=8&page=show_problem&category=&problem=1369&mosmsg=Submission+received+with+ID+14074742)

题意：
给定一个一元n次方程，求出所有解。(由代数基本定理可知n次一元多项式总是有n个根)

思路：利用牛顿迭代法 $x_{n+1}=x_n−\frac{f(x_n)}{f^′(x_n)}$，不断迭代就能求出较为精确的值，然后因为有的方程可能有多解，每次解得一个X后，就把原式子除以$(x-X)$，这个是肯定能整除的。把方程降阶然后继续用牛顿迭代法直到求出全部解

**公式简单推导**

$a_0+a_1x + a_2x^2 = (b_0 + b_1x)(x-x_0)=-b_0x_0+(b_0-b_1x_0)x+b_1x^2$

$a_2=b_1,a_1=b_0-b_1x_0=b_0-a_2x_0,a_0=-b_0*x_0$

$b_1=a_2$

$b_0=a_1+a_2x_0$

### 更高阶多项式也类似

In [39]:
maxn=10

def div (f, x, n):
    f[n+1] = 0
    for i in range(n,-1,-1):
        f[i] += f[i+1] * x

    for i in range(n):
        f[i] = f[i+1]
    return

def func (f, x, n):
    ret = 0
    u = 1
    for i in range(n+1):
        ret += f[i] * u
        u *= x
    return ret

def fx_newton(f,n):
    fd=[0]*maxn
    #计算导数f方程系数
    for i in range(n):
        fd[i] = f[i+1] * (i+1)
    x = -100
    eps=1e-16
    while True:
        nx = x - func(f, x, n) / func(fd, x, n-1)
        if abs(x-nx)<eps:
            break
        x=nx
    return x

def solve(N,a):
    cas = 1
    for i in range(N):
        x=fx_newton(a,N-i)
        print(x)
        div(a,x,N-i)

N=2
a=[0]*maxn
a[0]=-1
a[1]=0
a[2]=1
solve(N,a)

-1.0
1.0
